In [2]:
import numpy as np
import pandas as pd
from multiprocessing import Pool, cpu_count
NTHREAD = cpu_count()
from itertools import combinations
from tqdm import tqdm
import sys
argv = sys.argv
import os, utils, gc
#utils.start(__file__)
pd.set_option('display.max_columns', 99)
pd.set_option('display.max_rows', 200)
#==============================================================================

In [3]:
def f1(df):
            df['CODE_GENDER'] = 1 - (df['CODE_GENDER']=='F')*1 # 4 'XNA' are converted to 'M'
            df['FLAG_OWN_CAR'] = (df['FLAG_OWN_CAR']=='Y')*1
            df['FLAG_OWN_REALTY'] = (df['FLAG_OWN_REALTY']=='Y')*1
            df['EMERGENCYSTATE_MODE'] = (df['EMERGENCYSTATE_MODE']=='Yes')*1

            df['AMT_CREDIT-d-AMT_INCOME_TOTAL']   = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
            df['AMT_ANNUITY-d-AMT_INCOME_TOTAL']  = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
            df['AMT_GOODS_PRICE-d-AMT_INCOME_TOTAL']  = df['AMT_GOODS_PRICE'] / df['AMT_INCOME_TOTAL']

            df['AMT_CREDIT-d-AMT_ANNUITY']  = df['AMT_CREDIT'] / df['AMT_ANNUITY'] # how long should user pay?(month)
            df['AMT_GOODS_PRICE-d-AMT_ANNUITY'] = df['AMT_GOODS_PRICE'] / df['AMT_ANNUITY']# how long should user pay?(month)
            df['AMT_GOODS_PRICE-d-AMT_CREDIT']  = df['AMT_GOODS_PRICE'] / df['AMT_CREDIT']
            df['AMT_GOODS_PRICE-m-AMT_CREDIT']  = df['AMT_GOODS_PRICE'] - df['AMT_CREDIT']

            df['AMT_GOODS_PRICE-m-AMT_CREDIT-d-AMT_INCOME_TOTAL'] = df['AMT_GOODS_PRICE-m-AMT_CREDIT'] / df['AMT_INCOME_TOTAL']

            df['age_finish_payment'] = df['DAYS_BIRTH'].abs() + (df['AMT_CREDIT-d-AMT_ANNUITY']*30)
#            df['age_finish_payment'] = (df['DAYS_BIRTH']/-365) + df['credit-d-annuity']
            df.loc[df['DAYS_EMPLOYED']==365243, 'DAYS_EMPLOYED'] = np.nan
            df['DAYS_EMPLOYED-m-DAYS_BIRTH']                 = df['DAYS_EMPLOYED'] - df['DAYS_BIRTH']
            df['DAYS_REGISTRATION-m-DAYS_BIRTH']             = df['DAYS_REGISTRATION'] - df['DAYS_BIRTH']
            df['DAYS_ID_PUBLISH-m-DAYS_BIRTH']               = df['DAYS_ID_PUBLISH'] - df['DAYS_BIRTH']
            df['DAYS_LAST_PHONE_CHANGE-m-DAYS_BIRTH']        = df['DAYS_LAST_PHONE_CHANGE'] - df['DAYS_BIRTH']
            df['DAYS_REGISTRATION-m-DAYS_EMPLOYED']          = df['DAYS_REGISTRATION'] - df['DAYS_EMPLOYED']
            df['DAYS_ID_PUBLISH-m-DAYS_EMPLOYED']            = df['DAYS_ID_PUBLISH'] - df['DAYS_EMPLOYED']
            df['DAYS_LAST_PHONE_CHANGE-m-DAYS_EMPLOYED']     = df['DAYS_LAST_PHONE_CHANGE'] - df['DAYS_EMPLOYED']
            df['DAYS_ID_PUBLISH-m-DAYS_REGISTRATION']        = df['DAYS_ID_PUBLISH'] - df['DAYS_REGISTRATION']
            df['DAYS_LAST_PHONE_CHANGE-m-DAYS_REGISTRATION'] = df['DAYS_LAST_PHONE_CHANGE'] - df['DAYS_REGISTRATION']
            df['DAYS_LAST_PHONE_CHANGE-m-DAYS_ID_PUBLISH']   = df['DAYS_LAST_PHONE_CHANGE'] - df['DAYS_ID_PUBLISH']

            col = ['DAYS_EMPLOYED-m-DAYS_BIRTH',
                   'DAYS_REGISTRATION-m-DAYS_BIRTH',
                   'DAYS_ID_PUBLISH-m-DAYS_BIRTH',
                   'DAYS_LAST_PHONE_CHANGE-m-DAYS_BIRTH',
                   'DAYS_REGISTRATION-m-DAYS_EMPLOYED',
                   'DAYS_ID_PUBLISH-m-DAYS_EMPLOYED',
                   'DAYS_LAST_PHONE_CHANGE-m-DAYS_EMPLOYED',
                   'DAYS_ID_PUBLISH-m-DAYS_REGISTRATION',
                   'DAYS_LAST_PHONE_CHANGE-m-DAYS_REGISTRATION',
                   'DAYS_LAST_PHONE_CHANGE-m-DAYS_ID_PUBLISH'
                   ]
            col_comb = list(combinations(col, 2))

            for i,j in col_comb:
                df[f'{i}-d-{j}'] = df[i] / df[j]


            df['DAYS_EMPLOYED-d-DAYS_BIRTH']                 = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
            df['DAYS_REGISTRATION-d-DAYS_BIRTH']             = df['DAYS_REGISTRATION'] / df['DAYS_BIRTH']
            df['DAYS_ID_PUBLISH-d-DAYS_BIRTH']               = df['DAYS_ID_PUBLISH'] / df['DAYS_BIRTH']
            df['DAYS_LAST_PHONE_CHANGE-d-DAYS_BIRTH']        = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
            df['DAYS_REGISTRATION-d-DAYS_EMPLOYED']          = df['DAYS_REGISTRATION'] / df['DAYS_EMPLOYED']
            df['DAYS_ID_PUBLISH-d-DAYS_EMPLOYED']            = df['DAYS_ID_PUBLISH'] / df['DAYS_EMPLOYED']
            df['DAYS_LAST_PHONE_CHANGE-d-DAYS_EMPLOYED']     = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']
            df['DAYS_ID_PUBLISH-d-DAYS_REGISTRATION']        = df['DAYS_ID_PUBLISH'] / df['DAYS_REGISTRATION']
            df['DAYS_LAST_PHONE_CHANGE-d-DAYS_REGISTRATION'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_REGISTRATION']
            df['DAYS_LAST_PHONE_CHANGE-d-DAYS_ID_PUBLISH']   = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_ID_PUBLISH']

            df['OWN_CAR_AGE-d-DAYS_BIRTH'] = (df['OWN_CAR_AGE']*(-365)) / df['DAYS_BIRTH']
            df['OWN_CAR_AGE-m-DAYS_BIRTH'] = df['DAYS_BIRTH'] + (df['OWN_CAR_AGE']*365)
            df['OWN_CAR_AGE-d-DAYS_EMPLOYED'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
            df['OWN_CAR_AGE-m-DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'] + (df['OWN_CAR_AGE']*365)

            df['cnt_adults'] = df['CNT_FAM_MEMBERS'] - df['CNT_CHILDREN']
            df['CNT_CHILDREN-d-CNT_FAM_MEMBERS'] = df['CNT_CHILDREN'] / df['CNT_FAM_MEMBERS']
            df['income_per_adult'] = df['AMT_INCOME_TOTAL'] / df['cnt_adults']
#            df.loc[df['CNT_CHILDREN']==0, 'CNT_CHILDREN'] = np.nan
            df['AMT_INCOME_TOTAL-d-CNT_CHILDREN']       = df['AMT_INCOME_TOTAL']  / (df['CNT_CHILDREN']+0.000001)
            df['AMT_CREDIT-d-CNT_CHILDREN']       = df['AMT_CREDIT']        / (df['CNT_CHILDREN']+0.000001)
            df['AMT_ANNUITY-d-CNT_CHILDREN']      = df['AMT_ANNUITY']       / (df['CNT_CHILDREN']+0.000001)
            df['AMT_GOODS_PRICE-d-CNT_CHILDREN']  = df['AMT_GOODS_PRICE']   / (df['CNT_CHILDREN']+0.000001)
            df['AMT_INCOME_TOTAL-d-cnt_adults']       = df['AMT_INCOME_TOTAL']  / df['cnt_adults']
            df['AMT_CREDIT-d-cnt_adults']       = df['AMT_CREDIT']        / df['cnt_adults']
            df['AMT_ANNUITY-d-cnt_adults']      = df['AMT_ANNUITY']       / df['cnt_adults']
            df['AMT_GOODS_PRICE-d-cnt_adults']  = df['AMT_GOODS_PRICE']   / df['cnt_adults']
            df['AMT_INCOME_TOTAL-d-CNT_FAM_MEMBERS'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
            df['AMT_CREDIT-d-CNT_FAM_MEMBERS']       = df['AMT_CREDIT']       / df['CNT_FAM_MEMBERS']
            df['AMT_ANNUITY-d-CNT_FAM_MEMBERS']      = df['AMT_ANNUITY']      / df['CNT_FAM_MEMBERS']
            df['AMT_GOODS_PRICE-d-CNT_FAM_MEMBERS']  = df['AMT_GOODS_PRICE']  / df['CNT_FAM_MEMBERS']

            # EXT_SOURCE_x
            df['EXT_SOURCES_prod']  = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
            df['EXT_SOURCES_sum']  = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].sum(axis=1)
            df['EXT_SOURCES_sum']  = df['EXT_SOURCES_sum'].fillna(df['EXT_SOURCES_sum'].mean())
            df['EXT_SOURCES_mean']  = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
            df['EXT_SOURCES_mean']  = df['EXT_SOURCES_mean'].fillna(df['EXT_SOURCES_mean'].mean())
            df['EXT_SOURCES_std']   = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
            df['EXT_SOURCES_std']   = df['EXT_SOURCES_std'].fillna(df['EXT_SOURCES_std'].mean())

            df['EXT_SOURCES_1-2-3']  = df['EXT_SOURCE_1'] - df['EXT_SOURCE_2'] - df['EXT_SOURCE_3']
            df['EXT_SOURCES_2-1-3']  = df['EXT_SOURCE_2'] - df['EXT_SOURCE_1'] - df['EXT_SOURCE_3']
            df['EXT_SOURCES_1-2']    = df['EXT_SOURCE_1'] - df['EXT_SOURCE_2']
            df['EXT_SOURCES_2-3']    = df['EXT_SOURCE_2'] - df['EXT_SOURCE_3']
            df['EXT_SOURCES_1-3']    = df['EXT_SOURCE_1'] - df['EXT_SOURCE_3']


            # =========
            # https://www.kaggle.com/jsaguiar/updated-0-792-lb-lightgbm-with-simple-features/code
            # =========
            df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
            df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
            df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']

            # =========
            # https://www.kaggle.com/poohtls/fork-of-fork-lightgbm-with-simple-features/code
            # =========
            docs = [_f for _f in df.columns if 'FLAG_DOC' in _f]
            live = [_f for _f in df.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]
            inc_by_org = df[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']

            df['alldocs_kurt'] = df[docs].kurtosis(axis=1)
            df['alldocs_skew'] = df[docs].skew(axis=1)
            df['alldocs_mean'] = df[docs].mean(axis=1)
            df['alldocs_sum']  = df[docs].sum(axis=1)
            df['alldocs_std']  = df[docs].std(axis=1)
            df['NEW_LIVE_IND_SUM'] = df[live].sum(axis=1)
            df['NEW_INC_PER_CHLD'] = df['AMT_INCOME_TOTAL'] / (1 + df['CNT_CHILDREN'])
            df['NEW_INC_BY_ORG'] = df['ORGANIZATION_TYPE'].map(inc_by_org)
            df['NEW_ANNUITY_TO_INCOME_RATIO'] = df['AMT_ANNUITY'] / (1 + df['AMT_INCOME_TOTAL'])
            df['NEW_CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
            df['NEW_CAR_TO_EMPLOY_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
            df['NEW_PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
            df['NEW_PHONE_TO_EMPLOYED_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']


            # =============================================================================
            # Maxwell features
            # =============================================================================
            bdg_avg  = df.filter(regex='_AVG$').columns
            bdg_mode = df.filter(regex='_MODE$').columns
            bdg_medi = df.filter(regex='_MEDI$').columns[:len(bdg_avg)] # ignore FONDKAPREMONT_MODE...

            df['building_score_avg_mean'] = df[bdg_avg].mean(1)
            df['building_score_avg_std']  = df[bdg_avg].std(1)
            df['building_score_avg_sum']  = df[bdg_avg].sum(1)

            df['building_score_mode_mean'] = df[bdg_mode].mean(1)
            df['building_score_mode_std']  = df[bdg_mode].std(1)
            df['building_score_mode_sum']  = df[bdg_mode].sum(1)

            df['building_score_medi_mean'] = df[bdg_medi].mean(1)
            df['building_score_medi_std']  = df[bdg_medi].std(1)
            df['building_score_medi_sum']  = df[bdg_medi].sum(1)


            df['maxwell_feature_1'] = (df['EXT_SOURCE_1'] * df['EXT_SOURCE_3']) ** (1 / 2)



            df.replace(np.inf, np.nan, inplace=True) # TODO: any other plan?
            df.replace(-np.inf, np.nan, inplace=True)
            return

train = pd.read_csv('../input/application_train.csv')
f1(train)
# utils.to_pickles(train, '../data/train', utils.SPLIT_SIZE)
# utils.to_pickles(train[['TARGET']], '../data/label', utils.SPLIT_SIZE)

test = pd.read_csv('../input/application_test.csv')
f1(test)
# utils.to_pickles(test, '../data/test', utils.SPLIT_SIZE)
# test[['SK_ID_CURR']].to_pickle('../data/sub.p')

In [4]:
train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,...,CNT_CHILDREN-d-CNT_FAM_MEMBERS,income_per_adult,AMT_INCOME_TOTAL-d-CNT_CHILDREN,AMT_CREDIT-d-CNT_CHILDREN,AMT_ANNUITY-d-CNT_CHILDREN,AMT_GOODS_PRICE-d-CNT_CHILDREN,AMT_INCOME_TOTAL-d-cnt_adults,AMT_CREDIT-d-cnt_adults,AMT_ANNUITY-d-cnt_adults,AMT_GOODS_PRICE-d-cnt_adults,AMT_INCOME_TOTAL-d-CNT_FAM_MEMBERS,AMT_CREDIT-d-CNT_FAM_MEMBERS,AMT_ANNUITY-d-CNT_FAM_MEMBERS,AMT_GOODS_PRICE-d-CNT_FAM_MEMBERS,EXT_SOURCES_prod,EXT_SOURCES_sum,EXT_SOURCES_mean,EXT_SOURCES_std,EXT_SOURCES_1-2-3,EXT_SOURCES_2-1-3,EXT_SOURCES_1-2,EXT_SOURCES_2-3,EXT_SOURCES_1-3,DAYS_EMPLOYED_PERC,INCOME_PER_PERSON,PAYMENT_RATE,alldocs_kurt,alldocs_skew,alldocs_mean,alldocs_sum,alldocs_std,NEW_LIVE_IND_SUM,NEW_INC_PER_CHLD,NEW_INC_BY_ORG,NEW_ANNUITY_TO_INCOME_RATIO,NEW_CAR_TO_BIRTH_RATIO,NEW_CAR_TO_EMPLOY_RATIO,NEW_PHONE_TO_BIRTH_RATIO,NEW_PHONE_TO_EMPLOYED_RATIO,building_score_avg_mean,building_score_avg_std,building_score_avg_sum,building_score_mode_mean,building_score_mode_std,building_score_mode_sum,building_score_medi_mean,building_score_medi_std,building_score_medi_sum,maxwell_feature_1
0,100002,1,Cash loans,1,0,1,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637.0,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,...,0.0,202500.0,2.025000e+11,4.065975e+11,2.470050e+10,3.510000e+11,202500.0,406597.50,24700.50,351000.0,202500.0,406597.50,24700.50,351000.0,0.003043,0.485361,0.161787,0.092026,-0.319287,0.040536,-0.179912,0.123573,-0.056339,0.067329,202500.0,0.060749,20.0,4.472136,0.05,1,0.223607,5,202500.0,157500.0,0.121977,NaN,NaN,0.119860,1.780220,0.144336,0.286699,2.0207,0.128494,0.272671,2.0559,0.144814,0.287301,2.0274,0.107579
1,100003,0,Cash loans,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188.0,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,...,0.0,135000.0,2.700000e+11,1.293502e+12,3.569850e+10,1.129500e+12,135000.0,646751.25,17849.25,564750.0,135000.0,646751.25,17849.25,564750.0,NaN,0.933513,0.466757,0.219895,NaN,NaN,-0.310978,NaN,NaN,0.070862,135000.0,0.027598,20.0,4.472136,0.05,1,0.223607,4,270000.0,135000.0,0.132216,NaN,NaN,0.049389,0.696970,0.206343,0.308426,2.8888,0.183981,0.295250,2.9437,0.206814,0.308690,2.8954,NaN
2,100004,0,Revolving loans,1,1,1,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225.0,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,...,0.0,67500.0,6.750000e+10,1.350000e+11,6.750000e+09,1.350000e+11,67500.0,135000.00,6750.00,135000.0,67500.0,135000.00,6750.00,135000.0,NaN,1.285479,0.642739,0.122792,NaN,NaN,NaN,-0.173655,NaN,0.011814,67500.0,0.050000,0.0,0.000000,0.00,0,0.000000,7,67500.0,135000.0,0.099999,-0.001365,-0.115556,0.042791,3.622222,NaN,NaN,0.0000,

In [5]:
test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,...,CNT_CHILDREN-d-CNT_FAM_MEMBERS,income_per_adult,AMT_INCOME_TOTAL-d-CNT_CHILDREN,AMT_CREDIT-d-CNT_CHILDREN,AMT_ANNUITY-d-CNT_CHILDREN,AMT_GOODS_PRICE-d-CNT_CHILDREN,AMT_INCOME_TOTAL-d-cnt_adults,AMT_CREDIT-d-cnt_adults,AMT_ANNUITY-d-cnt_adults,AMT_GOODS_PRICE-d-cnt_adults,AMT_INCOME_TOTAL-d-CNT_FAM_MEMBERS,AMT_CREDIT-d-CNT_FAM_MEMBERS,AMT_ANNUITY-d-CNT_FAM_MEMBERS,AMT_GOODS_PRICE-d-CNT_FAM_MEMBERS,EXT_SOURCES_prod,EXT_SOURCES_sum,EXT_SOURCES_mean,EXT_SOURCES_std,EXT_SOURCES_1-2-3,EXT_SOURCES_2-1-3,EXT_SOURCES_1-2,EXT_SOURCES_2-3,EXT_SOURCES_1-3,DAYS_EMPLOYED_PERC,INCOME_PER_PERSON,PAYMENT_RATE,alldocs_kurt,alldocs_skew,alldocs_mean,alldocs_sum,alldocs_std,NEW_LIVE_IND_SUM,NEW_INC_PER_CHLD,NEW_INC_BY_ORG,NEW_ANNUITY_TO_INCOME_RATIO,NEW_CAR_TO_BIRTH_RATIO,NEW_CAR_TO_EMPLOY_RATIO,NEW_PHONE_TO_BIRTH_RATIO,NEW_PHONE_TO_EMPLOYED_RATIO,building_score_avg_mean,building_score_avg_std,building_score_avg_sum,building_score_mode_mean,building_score_mode_std,building_score_mode_sum,building_score_medi_mean,building_score_medi_std,building_score_medi_sum,maxwell_feature_1
0,100001,Cash loans,0,0,1,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.018850,-19241,-2329.0,-5170.0,-812,NaN,1,1,0,1,0,1,NaN,2.0,2,2,TUESDAY,18,0,0,0,0,0,0,Kindergarten,0.752614,0.789654,0.159520,0.0660,0.0590,0.9732,NaN,NaN,NaN,...,0.000000,67500.0,1.350000e+11,5.688000e+11,2.056050e+10,4.500000e+11,67500.0,284400.0,10280.25,225000.0,67500.0,284400.0,10280.250,225000.0,0.094803,1.701788,0.567263,0.353601,-0.196559,-0.122480,-0.037040,0.630135,0.593095,0.121044,67500.0,0.036147,20.0,4.472136,0.05,1,0.223607,5,135000.0,135000.0,0.152299,NaN,NaN,0.090432,0.747102,0.235267,0.363330,1.4116,0.182037,0.322761,1.4563,0.235517,0.363183,1.4131,0.346492
1,100005,Cash loans,1,0,1,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.035792,-18064,-4469.0,-9118.0,-1623,NaN,1,1,0,1,0,0,Low-skill Laborers,2.0,2,2,FRIDAY,9,0,0,0,0,0,0,Self-employed,0.564990,0.291656,0.432962,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,49500.0,9.900000e+10,2.227680e+11,1.737000e+10,1.800000e+11,49500.0,111384.0,8685.00,90000.0,49500.0,111384.0,8685.000,90000.0,0.071345,1.289607,0.429869,0.136694,-0.159627,-0.706296,0.273335,-0.141306,0.132029,0.247398,49500.0,0.077973,20.0,4.472136,0.05,1,0.223607,4,99000.0,157500.0,0.175453,NaN,NaN,-0.000000,-0.000000,NaN,NaN,0.0000,0.000000,NaN,0.0000,NaN,NaN,0.0000,0.494590
2,100013,Cash loans,1,1,1,0,202500.0,663264.0,69777.0,630000.0,NaN,Working,Higher education,Married,House / apartment,0.019101,-20038,-4458.0,-2175.0,-3503,5.0,1,1,0,1,0,0,Drivers,2.0,2,2,MONDAY,14,0,0,0,0,0,0,Transport: type 3,NaN,0.699787,0.610991,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,101250.0,2.025000e+11,6.632640e+11,6.977700e+10,6.300000e+11,101250.0,331632.0,34888.50,315000.0,101250.0,331632.0,34888.500,315000.0,NaN,1.310778,0.655389,0.062788,NaN,NaN,NaN,0.088796,NaN,0.222477,101250.0,0.105202,20.0,4.472136,0.05,1,0.223607,5,202500.0,180000.0,0.344576,-0.000250,-0.001122,0.042719,0.192014,NaN,NaN,0.0000,0.000000,NaN,0.0000,NaN,NaN,0.0000,N

In [13]:
train['CODE_GENDER']==1

0          True
1         False
2          True
3         False
4          True
          ...  
307506     True
307507    False
307508    False
307509    False
307510    False
Name: CODE_GENDER, Length: 307511, dtype: bool

In [ ]:
import 